In [34]:
# import the dependencies

import pandas as pd
import numpy as np

from math import radians, degrees, sin, cos, asin, acos, sqrt, pi

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [35]:
def great_circle_distance(lon1, lat1, lon2, lat2, miles=False):
    """
    Compute the Great Circle Distance between two points given by longitude and latitude, in degrees,
    on the surface of the Earth.   Code is modified from:  
    https://medium.com/@petehouston/calculate-distance-of-two-locations-on-earth-using-python-1501b1944d97
    """
    # Radius of the Earth
    if miles:
        multiplier = 3958.756
    else:
        multiplier = 6378.137
        
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    return multiplier * (
        acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon1 - lon2))
    )

In [36]:
great_circle_distance(-71.312796, 41.49008,  -81.695391, 41.499498)

865.1813931328801

In [37]:
great_circle_distance(-71.312796, 41.49008,  -81.695391, 41.499498, miles=True)

536.9972503182587

In [38]:
def bounding_box(lon, lat, distance, miles=False):
    """
    Return bounding coordinates that includes all points within the given distance of the point given by lon, lat.
    Based on the code given by Jan Philip Matuschek at:
    http://janmatuschek.de/LatitudeLongitudeBoundingCoordinates
    """
    
    # Radius of the Earth
    if miles:
        multiplier = 3958.756
    else:
        multiplier = 6378.137
    
    lon, lat = map(radians, [lon, lat])
    
    # angular distance in radians on a great circle
    rad_dist = distance / multiplier;

    min_lat = lat - rad_dist
    max_lat = lat + rad_dist
    
    if min_lat > radians(-90) and max_lat < radians(90):
        delta_lon = asin(sin(rad_dist) / cos(lat))
        
        min_lon = lon - delta_lon
        if min_lon < radians(-180):
            min_lon += 2 * pi
            
        max_lon = lon + delta_lon
        if max_lon > radians(180):
            max_lon -= 2 * pi
    
    else:
        # a pole is within the distance
        min_lat = max(min_lat, radians(-90))
        max_lat = min(max_lat, radians(90))
        min_lon = radians(-180)
        max_lon = radians(180)

    return map(degrees, [min_lon, max_lon, min_lat, max_lat])


In [39]:
min_lon, max_lon, min_lat, max_lat = bounding_box(-93.2650, 44.9778, 50)
print(min_lon, max_lon, min_lat, max_lat)

-93.89996534735104 -92.63003465264896 44.52864235794024 45.42695764205976


In [40]:
great_circle_distance(min_lon, min_lat, -93.2650, 44.9778)

70.84892771381021

In [41]:
# Connect to the GHCN stations database
engine = create_engine('sqlite:///static/data/ghcn.db')

In [42]:
# Declare a Base using `automap_base()`
Base = automap_base()

In [43]:
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [44]:
# List the tables in the database
Base.classes.keys()

['countries', 'inventory', 'stations', 'states']

In [45]:
# Save references to each table
Inventory = Base.classes.inventory
Stations = Base.classes.stations

In [46]:
# Create a session
session = Session(engine)

In [69]:
def find_stations_with(session, element, lon_min, lon_max, lat_min, lat_max, first_year=None, last_year=None,):
    """
    Args:  
        element: one of the elements in the inventory table.
        first_year: the earliest year that must be included.  If None, we do not filter on first_year
        last_year: the latest year that must be included.  If None, we do not filter on last_year
    """
    
    if first_year:
        if last_year: 
            t = session.query(
                Inventory.station_id,
             ).filter(Inventory.first_year <= first_year)\
            .filter(Inventory.last_year >= last_year)\
            .filter(Inventory.element == element).subquery('t')
        else:
            t = session.query(
                Inventory.station_id,
            ).filter(Inventory.first_year <= first_year)\
            .filter(Inventory.element == element).subquery('t')
    else:
        if last_year: 
            t = session.query(
                Inventory.station_id,
            ).filter(Inventory.last_year >= last_year)\
            .filter(Inventory.element == element).subquery('t')
        else:
            t = session.query(
                Inventory.station_id,
            ).filter(Inventory.element == element).subquery('t')
        

    query = session.query(Stations).filter(Stations.station_id == t.c.station_id)\
    .filter(Stations.longitude >= lon_min)\
    .filter(Stations.longitude <= lon_max)\
    .filter(Stations.latitude >= lat_min)\
    .filter(Stations.latitude <= lat_max)

    return pd.read_sql(query.statement, session.bind)

In [70]:
# Look at the first row of the inventory table
first_row = session.query(Inventory).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1bce3880cf8>,
 'last_year': 1949,
 'element': 'TMAX',
 'id': 1,
 'first_year': 1949,
 'station_id': 'ACW00011604'}

In [71]:
def find_stations_near(session, lon, lat, radius, element, miles=False, first_year=None, last_year=None):
    # First get the bounding box to limit the search:
    lon_min, lon_max, lat_min, lat_max = bounding_box(lon, lat, radius, miles)
    
    # now query the database for all points within bounding box 
    df = find_stations_with(session, element, lon_min, lon_max, lat_min, lat_max, first_year, last_year)
    
    distance = []
    # now compute the great circle distance to each station
    for row in df.itertuples():
        distance.append(great_circle_distance(lon, lat, row.longitude, row.latitude, miles))

    df['distance'] = distance
    
    return df[df.distance <= radius]
    

In [72]:
# How does this behave with empty search results?  seems ok
find_stations_near(session, -93.2650, 44.9778, 0.5, 'PRCP')

,station_id,latitude,longitude,elevation,country_id,state_id,name,gsn,hcn,wmo,distance


In [73]:
find_stations_near(session, -93.2650, 44.9778, 5, 'PRCP')

,station_id,latitude,longitude,elevation,country_id,state_id,name,gsn,hcn,wmo,distance
0,US1MNHN0009,45.0039,-93.2895,272.2,US,MN,MINNEAPOLIS 3.0 NNW,None,None,None,3.487394
1,US1MNHN0025,44.9817,-93.2783,249.9,US,MN,MINNEAPOLIS TARGET FIELD,None,None,None,1.133698
3,US1MNHN0065,45.0083,-93.2363,266.4,US,MN,MINNEAPOLIS 2.4 NE,None,None,None,4.078298
4,US1MNHN0093,44.9417,-93.2418,261.5,US,MN,MINNEAPOLIS 1.9 SE,None,None,None,4.414640
6,US1MNHN0147,44.9516,-93.3013,267.3,US,MN,MINNEAPOLIS 1.8 WSW,None,None,None,4.084222
7,US1MNHN0160,44.9553,-93.2176,254.2,US,MN,MINNEAPOLIS 2.5 ESE,None,None,None,4.495628
8,US1MNHN0163,44.9440,-93.2432,264.6,US,MN,MINNEAPOLIS 1.7 SE,None,None,None,4.135911
9,US1MNHN0169,44.9510,-93.2175,257.6,US,MN,MINNEAPOLIS 2.6 ESE,None,None,None,4.785142
11,US1MNHN0181,44.9671,-93.2622,258.5,US,MN,MINNEAPOLIS 0.4 NE,None,None,None,1.211357
12,USC00214884,44.9783,-93.2469,229.8,US,MN,LOWER ST ANTHONY FALLS,None,None,None,1.426369


In [76]:
find_stations_near(session, -93.2650, 44.9778, 50, 'SNOW', miles=False, first_year=2015, last_year=2019)

,station_id,latitude,longitude,elevation,country_id,state_id,name,gsn,hcn,wmo,distance
0,US1MNAA0004,45.4090,-93.3090,284.1,US,MN,SAINT FRANCIS 4.0 E,None,None,NaN,48.124909
1,US1MNAA0005,45.3846,-93.1553,280.4,US,MN,EAST BETHEL 3.1 NE,None,None,NaN,46.095562
2,US1MNAA0006,45.1708,-93.2570,277.1,US,MN,BLAINE 2.4 W,None,None,NaN,21.493864
3,US1MNAA0013,45.1648,-93.2855,267.6,US,MN,COON RAPIDS 1.4 ESE,None,None,NaN,20.879038
4,US1MNAA0020,45.1646,-93.2072,276.5,US,MN,BLAINE 0.4 S,None,None,NaN,21.285178
5,US1MNAA0031,45.2488,-93.1597,274.9,US,MN,HAM LAKE 2.2 E,None,None,NaN,31.281188
6,US1MNAA0036,45.2582,-93.4114,268.5,US,MN,RAMSEY 1.9 E,None,None,NaN,33.265052
7,US1MNAA0054,45.1947,-93.3749,269.1,US,MN,ANOKA 1.3 SSE,None,None,NaN,25.643721
10,US1MNCG0011,45.3318,-92.8982,275.8,US,MN,CHISAGO CITY 1.3 SSE,None,None,NaN,48.806031
11,US1MNCV0001,44.7573,-93.6416,258.2,US,MN,CARVER 0.7 W,None,None,NaN,38.539929
